# SQL Case Study

Use sqlalchemy to load the SQLite database.

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
engine = create_engine('sqlite:///country_club.db')
engine.table_names()

['Bookings', 'Facilities', 'Members']

Connect to the SQLite database

In [7]:
connection = engine.connect()

### Questions

/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [8]:
query = connection.execute("""
SELECT *
FROM (
    SELECT name AS facility,
        SUM(CASE WHEN memid = 0 THEN guestcost * slots
            ELSE membercost * slots END) AS revenues
    FROM Bookings
    INNER JOIN Facilities
        USING (facid)
    GROUP BY facid
    ORDER BY revenues
)
WHERE revenues < 1000
""")
df = pd.DataFrame(query.fetchall(), columns=query.keys())
df

,facility,revenues
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [10]:
query = connection.execute("""
SELECT M.surname, M.firstname, R.surname || ', ' || R.firstname
FROM Members AS M
INNER JOIN Members AS R
    ON M.recommendedby = R.memid
ORDER BY M.surname, M.firstname
""")
df = pd.DataFrame(query.fetchall(), columns=query.keys())
df

,surname,firstname,"R.surname || ', ' || R.firstname"
0,Bader,Florence,"Stibbons, Ponder"
1,Baker,Anne,"Stibbons, Ponder"
2,Baker,Timothy,"Farrell, Jemima"
3,Boothe,Tim,"Rownam, Tim"
4,Butters,Gerald,"Smith, Darren"
5,Coplin,Joan,"Baker, Timothy"
6,Crumpet,Erica,"Smith, Tracy"
7,Dare,Nancy,"Joplette, Janice"
8,Genting,Matthew,"Butters, Gerald"
9,Hunt,John,"Purview, Millicent"


/* Q12: Find the facilities with their usage by member, but not guests */

In [11]:
query = connection.execute("""
SELECT name AS facility, SUM(slots * 0.5) AS member_usage_hours
FROM Facilities
INNER JOIN Bookings
    USING (facid)
WHERE memid != 0
GROUP BY facid
ORDER BY facility
""")
df = pd.DataFrame(query.fetchall(), columns=query.keys())
df

,facility,member_usage_hours
0,Badminton Court,543.0
1,Massage Room 1,442.0
2,Massage Room 2,27.0
3,Pool Table,428.0
4,Snooker Table,430.0
5,Squash Court,209.0
6,Table Tennis,397.0
7,Tennis Court 1,478.5
8,Tennis Court 2,441.0


/* Q13: Find the facilities usage by month, but not guests */

In [ ]:
query = connection.execute("""
""")
df = pd.DataFrame(query.fetchall(), columns=query.keys())
df

Close the database engine connection

In [5]:
connection.close()